## Import Packages

In [ ]:
import os
import mediapipe as mp
import cv2
import time

## Import Model & Configurations

In [ ]:
# Path Model TF Lite
model_path = os.path.join(os.getcwd(), 'model/efficientnet_lite2_float.tflite')

# Model Config
BaseOptions = mp.tasks.BaseOptions
ImageClassifier = mp.tasks.vision.ImageClassifier
ImageClassifierOptions = mp.tasks.vision.ImageClassifierOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = ImageClassifierOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    max_results=5,
    running_mode=VisionRunningMode.IMAGE)

## Inference

In [ ]:
with ImageClassifier.create_from_options(options) as classifier:
    # Mengambil gambar dari Webcam
    cap = cv2.VideoCapture(0)

    # Error Handling Webcam tidak accessible
    if not cap.isOpened():
        print("Camera failed to open.")
        exit()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert ke RGB lalu Mediapipe Image
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # Menghitung Latensi & Melakukan Inference
        start_time = time.time()
        result = classifier.classify(mp_image)
        latency = (time.time() - start_time) * 1000  # ms

        # Menyimpan hasil klasifikasi dan menampilkannya di window
        if result and result.classifications:
            top_class = result.classifications[0].categories[0]
            label = f"{top_class.category_name} ({top_class.score:.2f})"
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Latency: {latency:.1f} ms", (10, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

        # Menampilkan frame di window
        cv2.imshow("Live Classification", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()